In [1]:
import matplotlib.pyplot as plt
import numpy as np
import copy
import h5py
import os
import seaborn as sns
from pyscf.scf.uhf import det_ovlp

In [138]:
num_det = 2

In [139]:
# Wavefunction generation
from pyscf import lib, gto, scf
from pyscf import gto, mp, mcscf
def He_casci(scf_checkfile, ci_checkfile):
    mol = gto.M(
        atom="He 0. 0. 0.0;",
        basis=f"ccecpccpvqz",
        unit="bohr",
        charge=0,
        spin=0,
        verbose=1,
    )
    mf = scf.UHF(mol).run()
    mf.chkfile = scf_checkfile
    mf.kernel()
    mc = mcscf.CASCI(mf, num_det,2)
    # mc.fcisolver.nroots = 2
    mc.kernel()
    print(mc.__dict__.keys())
    with h5py.File(ci_checkfile, "a") as f:
        f.create_group("ci")
        f["ci/ncas"] = mc.ncas
        f["ci/nelecas"] = list(mc.nelecas)
        f["ci/ci"] = mc.ci
        f["ci/mo_coeff"] = mc.mo_coeff    
    return mol, mf, mc

In [140]:
scf_checkfile = 'rohf.chk'
ci_checkfile = 'casci.chk'
for fname in [scf_checkfile, ci_checkfile]:
    if os.path.isfile(fname):
        os.remove(fname)
mol, mf, mcc = He_casci(scf_checkfile, ci_checkfile)


dict_keys(['mol', '_scf', 'verbose', 'stdout', 'max_memory', 'ncas', 'nelecas', '_ncore', 'fcisolver', 'frozen', 'extrasym', 'e_tot', 'e_cas', 'ci', 'mo_coeff', 'mo_energy', 'mo_occ', 'converged'])


In [141]:
mcc.ci

FCIvector([[ 0.99897607,  0.00661145],
           [ 0.00661145, -0.04426495]])

In [142]:
import pyqmc.api as pyq
from wftools import generate_slater

In [143]:
# Boson WF
import bosonwftools, jastrowspin, bosonslater
import pyscftools, mc
wf0, _ = bosonwftools.generate_boson_wf(mol, mf, mc=mcc)
wave_functions = wf0.wf_factors
for wave in wave_functions:
    if isinstance(wave, bosonslater.BosonWF):
        boson_wf = wave
    if isinstance(wave, jastrowspin.JastrowSpin):
        jastrow_wf = wave       
                


In [243]:
from wftools import generate_slater
wfs = []
num_det = mcc.ci.shape[0] * mcc.ci.shape[1]
for i in range(mcc.ci.shape[0]):
    for j in range(mcc.ci.shape[1]):
        mc0 = copy.copy(mcc)
        mc0.ci = mcc.ci * 0
        mc0.ci[i, j] = 1
        wf0, _ = generate_slater(mol, mf, mc=mc0, optimize_determinants=False)
        wfs.append(wf0)

In [244]:
mc0.ci

FCIvector([[0., 0.],
           [0., 1.]])

In [145]:
configs = pyq.initial_guess(mol, 1)
# configs.configs

In [245]:
configs.configs

array([[[-0.84833619, -0.60313611, -0.8611233 ],
        [-0.62588997, -0.40479264, -0.6075714 ]]])

In [255]:
# Boson grad
e = 0
boson_wf.recompute(configs)
g, _, _ = boson_wf.gradient_value(e, configs.electron(e))
gradb = mc.limdrift(np.real(g.T))
valueb = boson_wf.value()
gradb

array([[0.62068918, 0.44128738, 0.63004493]])

In [256]:
# Fermion grad
e = 0
_ = [wf.recompute(configs) for wf in wfs]
grads = [np.real(wf.gradient(e, configs.electron(e)).T) for wf in wfs]
vals = [np.real(wf.value()) for wf in wfs]
grad = mc.limdrift(np.mean(grads, axis=0))
grad

array([[-0.62796992, -0.44646373, -0.63743542]])

In [248]:
# For some configurations, grad and gradb are of opposite sign and close in absolute value, try a few times and find such configurations
# 1. Use the configuration coordinates above, and then use fixed initial configs. 

In [253]:
# Fermion grad enhanced
import gpu
s = int(e >= wfs[0]._nelec[0])
epos = configs.electron(e)
aograd = wfs[0].orbitals.aos("GTOval_sph_deriv1", epos)
mograd = wfs[0].orbitals.mos(aograd, s)
mograd_vals = mograd[:, :, wfs[0]._det_occup[s]]
vec = mograd_vals

ratios = gpu.cp.einsum(
    "ei...dj,idj...->ei...d",
    vec,
    wfs[0]._inverse[s][..., e - s * wfs[0]._nelec[0]],
)
ratios

array([[[ 1.        ,  1.        ]],

       [[ 0.98442461, -3.99581063]],

       [[ 0.69989002, -2.84087572]],

       [[ 0.99926301, -4.05604014]]])

In [232]:
# Boson grad enhanced
import gpu
s = int(e >= boson_wf._nelec[0])
epos = configs.electron(e)
aograd = boson_wf.orbitals.aos("GTOval_sph_deriv1", epos)
mograd = boson_wf.orbitals.mos(aograd, s)
mograd_vals = mograd[:, :, boson_wf._det_occup[s]]
vec = mograd_vals

ratios = gpu.cp.einsum(
    "ei...dj,idj...->ei...d",
    vec,
    boson_wf._inverse[s][..., e - s * boson_wf._nelec[0]],
)

upref = gpu.cp.amax(boson_wf._dets[0][1]).real
dnref = gpu.cp.amax(boson_wf._dets[1][1]).real

det_array = (
    boson_wf._dets[0][0, :, boson_wf._det_map[0]]
    * boson_wf._dets[1][0, :, boson_wf._det_map[1]]
    * gpu.cp.exp(
        boson_wf._dets[0][1, :, boson_wf._det_map[0]]
        + boson_wf._dets[1][1, :, boson_wf._det_map[1]]
        - upref
        - dnref
    )
)

numer = 1./2 * gpu.cp.einsum(
    "ei...d,di->ei...",
    ratios[..., boson_wf._det_map[s]],
    # boson_wf.parameters["det_coeff"],
    det_array**2,
    # det_array_noref,
)

numera = 1./2 * gpu.cp.einsum(
    "ei...d,di->ei...",
    ratios[..., boson_wf._det_map[s]],
    # boson_wf.parameters["det_coeff"],
    det_array,
    # det_array_noref,
)

denom = gpu.cp.einsum(
    "di->i...",
    # boson_wf.parameters["det_coeff"],
    det_array**2,
)

denoma = gpu.cp.einsum(
    "di->i...",
    # boson_wf.parameters["det_coeff"],
    det_array,
)

res = numer / denom
res[1:]/res[0]


array([[0.62068918],
       [0.44128738],
       [0.63004493]])

In [214]:
ratios[..., boson_wf._det_map[s]]

array([[[ 1.        ,  1.        ,  1.        ,  1.        ]],

       [[ 0.98442461,  0.98442461, -3.99581063, -3.99581063]],

       [[ 0.69989002,  0.69989002, -2.84087572, -2.84087572]],

       [[ 0.99926301,  0.99926301, -4.05604014, -4.05604014]]])

In [204]:
det_array

array([[ 1.        ],
       [-0.32715703],
       [ 0.28069611],
       [-0.09183171]])

In [206]:
numer

array([[0.14928189],
       [0.09265765],
       [0.06587621],
       [0.0940543 ]])

In [228]:
numera

array([[ 0.43085368],
       [-0.04615161],
       [-0.03281211],
       [-0.04684726]])

In [222]:
ratios[..., boson_wf._det_map[s]]

array([[[ 1.        ,  1.        ,  1.        ,  1.        ]],

       [[ 0.98442461,  0.98442461, -3.99581063, -3.99581063]],

       [[ 0.69989002,  0.69989002, -2.84087572, -2.84087572]],

       [[ 0.99926301,  0.99926301, -4.05604014, -4.05604014]]])

In [233]:
denoma

array([0.86170737])

In [225]:
upref

-2.060677852325388

In [226]:
dnref

-1.4388445176364784

In [166]:
boson_wf.parameters['det_coeff']

array([0.25, 0.25, 0.25, 0.25])

In [163]:
s = int(e >= boson_wf._nelec[0])
aograd = boson_wf.orbitals.aos("GTOval_sph_deriv1", configs.electron(e))
mograd = boson_wf.orbitals.mos(aograd, s)
mograd_vals = mograd[:, :, boson_wf._det_occup[s]]
ratios = boson_wf._testrowderiv(e, mograd_vals)
ratios

array([[0.5       ],
       [0.31034459],
       [0.22064369],
       [0.31502246]])

In [167]:
(ratios[1:] / ratios[0]).T

array([[0.62068918, 0.44128738, 0.63004493]])

In [121]:
s = int(e >= wf._nelec[0])
aograd = wf.orbitals.aos("GTOval_sph_deriv1", configs.electron(e))
mograd = wf.orbitals.mos(aograd, s)
mograd_vals = mograd[:, :, wf._det_occup[s]]
ratios = wf._testrowderiv(e, mograd_vals)
ratios

AttributeError: 'MultiplyWF' object has no attribute '_nelec'